# Evaluation of Reconstruction Performance

In [ ]:
import sys
import os
from os.path import join
parent_dir = os.path.abspath(join(os.getcwd(), os.pardir))
app_dir = join(parent_dir, "app")
if app_dir not in sys.path:
      sys.path.append(app_dir)

from pathlib import Path
import torch as pt
from torch.utils.data import Subset
from torch.nn.functional import mse_loss
from CNN_VAE import ConvEncoder, ConvDecoder, Autoencoder
from utils.training_loop import train_cnn_vae
import utils.config as config
import matplotlib.pyplot as plt
import importlib

importlib.reload(config)

pt.manual_seed(0)

plt.rcParams["figure.dpi"] = 180

# use GPU if possible
device = pt.device("cuda:0") if pt.cuda.is_available() else pt.device("cpu")
print(device)

TIMESTEP = (config.mini_test_per_cond - 1) if config.mini_dataset else config.timestep_reconstruction

DATA_PATH = Path(os.path.abspath('')).parent / "data"
OUTPUT_PATH = Path(os.path.abspath('')).parent / "output" / "VAE"
MODEL_PATH = Path(os.path.abspath('')).parent / "output" / "VAE" / "latent_study"

In [ ]:
# load test dataset
test_dataset = pt.load(join(DATA_PATH, "test_dataset.pt"))

# split test dataset into the two flow conditions
X_test_1 = Subset(test_dataset,                                 # ma0.84 alpha3.00 
                  list(range(0, int(len(test_dataset) / 2))))        
X_test_2 = Subset(test_dataset,                                 # ma0.84 alpha5.00
                  list(range(int(len(test_dataset) / 2), len(test_dataset))))    

# make tensors from datasets
X_test_1_tensor = pt.stack([X_test_1[n] for n in range(len(X_test_1))], dim=3).squeeze(0)
X_test_2_tensor = pt.stack([X_test_2[n] for n in range(len(X_test_2))], dim=3).squeeze(0)
print(X_test_1_tensor.shape)

#### MSE and Variance Reconstruction with varying number of bottleneck neurons

In [ ]:
# function to create VAE model
def make_VAE_model(n_latent: int = 256) -> pt.nn.Module:
    encoder = ConvEncoder(
        in_size=config.target_resolution,
        n_channels=config.input_channels,
        n_latent=n_latent,
        variational=True,
        layernorm=True
    )

    decoder = ConvDecoder(
        in_size=config.target_resolution,
        n_channels=config.output_channels,
        n_latent=n_latent,
        layernorm=True,
        squash_output=True
    )

    autoencoder = Autoencoder(encoder, decoder)
    autoencoder.to(device)
    return autoencoder

In [ ]:

# scan directory for trained models and extract paths as well as the latent size of the model
dirs = [os.path.join(MODEL_PATH, name, name) for name in os.listdir(MODEL_PATH) if os.path.isdir(os.path.join(MODEL_PATH, name))]
sorted_dirs = sorted(dirs, key=lambda x: int(os.path.basename(x)))
latent_sizes = [int(os.path.basename(dir)) for dir in sorted_dirs]

In [ ]:
# Initialize lists to save the computed metrics
MSE_1 = []
MSE_2 = []
Var1 = []
Var2 = []

# compute the total variance of test datasets
orig_Var1 = pt.var(X_test_1_tensor)
orig_Var2 = pt.var(X_test_2_tensor)

for i, latent_size in enumerate(latent_sizes):
    print("Computing metrics for autoencoder with latent size ", latent_size)
    # load model
    autoencoder = make_VAE_model(latent_size)
    autoencoder.load(sorted_dirs[i])
    autoencoder.eval()

    # reconstruct test dataset 1
    with pt.no_grad():
        reconstructed = pt.stack([autoencoder(X_test_1[n].unsqueeze(0)).squeeze(0).detach() for n in range(len(X_test_1))], dim=3).squeeze(0)
    
    print(reconstructed)
    # compute MSE
    MSE_1.append(mse_loss(X_test_1_tensor, reconstructed).item())

    # compute variance reconstruction
    Var1.append(((1 - ((orig_Var1 - pt.var(reconstructed)) / orig_Var1)) * 100).item())

    # reconstruct test dataset 2
    with pt.no_grad():
        reconstructed = pt.stack([autoencoder(X_test_2[n].unsqueeze(0)).squeeze(0).detach() for n in range(len(X_test_2))], dim=3).squeeze(0)

    # compute MSE
    MSE_2.append(mse_loss(X_test_2_tensor, reconstructed).item())

    # compute variance reconstruction
    Var2.append(((1 - ((orig_Var2 - pt.var(reconstructed)) / orig_Var2)) * 100).item())

In [ ]:
# Plot the results and save the figure
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(latent_sizes, MSE_1, label="Test Dataset 1")
ax1.plot(latent_sizes, MSE_2, label="Test Dataset 2")
ax1.set_title("MSE")
ax2.plot(latent_sizes, Var1, label="Test Dataset 1")
ax2.plot(latent_sizes, Var2, label="Test Dataset 2")
ax2.set_title("Variance Reconstruction in %")
ax2.set_xlabel("latent size")
ax2.set_xticks(range(0, 325, 25))
handles, labels = ax2.get_legend_handles_labels()
fig.legend(handles, labels)
fig.tight_layout()
fig.savefig(join(OUTPUT_PATH, "MSE_and_Variance_with_latent_size.png"), bbox_inches = "tight")

#### Temporal MSE distribution with varying number of bottleneck neurons

In [ ]:
timesteps = range(config.mini_test_per_cond) if config.mini_dataset else range(config.time_steps_per_cond)

fig, ax1 = plt.subplots(1, 1, figsize = (8, 3))
    
for i, latent_size in enumerate(latent_sizes):
    print("Computing metrics for autoencoder with latent size ", latent_size)
    # load model
    autoencoder = make_VAE_model(latent_size)
    autoencoder.load(sorted_dirs[i])
    autoencoder.eval()

    # reconstruct test dataset 1
    with pt.no_grad():
        reconstructed = pt.stack([autoencoder(X_test_1[n].unsqueeze(0)).squeeze(0).detach() for n in range(len(X_test_1))], dim=3).squeeze(0)

    MSE = ((X_test_1_tensor - reconstructed)**2).mean(dim=[0, 1])
    ax1.plot(timesteps, MSE, label="latent size {}".format(latent_size))

# ax1.set_title("Test Dataset 1")
ax1.set_ylabel("MSE")
ax1.set_xlabel("timestep")
# ax1.set_yscale("log")

fig.legend()
fig.tight_layout()
fig.savefig(join(OUTPUT_PATH, "temporal_MSE_distribution_VAE.png"), bbox_inches = "tight")


#### Spatial MSE distribution with varying number of bottleneck neurons

In [ ]:
# Load coordinates
coords = pt.load(join(DATA_PATH, "coords_interp.pt"))
xx, yy = coords

In [ ]:
fig, axes = plt.subplots(2, 3, sharey=True)

for i, latent_size in enumerate([10, 110, 210]):
    print("Computing metrics for autoencoder with latent size ", latent_size)
    # load model
    autoencoder = make_VAE_model(latent_size)
    autoencoder.load(sorted_dirs[latent_sizes.index(latent_size)])
    autoencoder.eval()

    # reconstruct test dataset 1
    with pt.no_grad():
        reconstructed = pt.stack([autoencoder(X_test_1[n].unsqueeze(0)).squeeze(0).detach() for n in range(len(X_test_1))], dim=3).squeeze(0)

    MSE1 = ((X_test_1_tensor - reconstructed)**2).mean(dim=2)

    # reconstruct test dataset 2
    with pt.no_grad():
        reconstructed = pt.stack([autoencoder(X_test_2[n].unsqueeze(0)).squeeze(0).detach() for n in range(len(X_test_2))], dim=3).squeeze(0)

    MSE2 = ((X_test_2_tensor - reconstructed)**2).mean(dim=2)

    # compute the mean MSE and the corresponding standard deviation for the lowest rank to compare to higher ranks
    if i == 0:
        mean, std = MSE1.mean(), MSE1.std()
        vmin, vmax = 0, mean + 1*std
        levels = pt.linspace(vmin, vmax, 120)

        axes[0][i].set_ylabel("Test Dataset 1")
        axes[1][i].set_ylabel("Test Dataset 2")

    # create the contour plot
    cont = axes[0][i].contourf(xx, yy, MSE1, vmin=vmin, vmax=vmax, levels=levels, extend="both")
    cont = axes[1][i].contourf(xx, yy, MSE2, vmin=vmin, vmax=vmax, levels=levels, extend="both")

    # formatting
    axes[0][i].set_title("latent size {}".format(latent_size))

    for row in range(2):
        axes[row][i].set_aspect("equal")
        axes[row][i].set_xticklabels([])
        axes[row][i].set_yticklabels([])

# add seperate subplot for color axis
fig.subplots_adjust(right=0.9)
cax = fig.add_axes([0.99, 0.042, 0.03, 0.885])
cbar = fig.colorbar(cont, cax=cax,label = "MSE")

fig.tight_layout()
fig.savefig(join(OUTPUT_PATH, "spatial_MSE_distribution_VAE.png"), bbox_inches = "tight")

#### Reconstructed pressure field compared to Ground Truth for two bottleneck sizes

In [ ]:
fig, axes = plt.subplots(2, 3, sharey=True)

for i, latent_size in enumerate([10, 210, "experimental"]):
    # create the contour plot
    if latent_size == "experimental":
        cont = axes[0][i].contourf(xx, yy, X_test_1[TIMESTEP].squeeze(0), vmin=vmin, vmax=vmax, levels=levels, extend="both")
        cont = axes[1][i].contourf(xx, yy, X_test_2[TIMESTEP].squeeze(0), vmin=vmin, vmax=vmax, levels=levels, extend="both")
        axes[0][i].set_title("Ground Truth")
    else:
        # load model
        autoencoder = make_VAE_model(latent_size)
        autoencoder.load(sorted_dirs[latent_sizes.index(latent_size)])
        autoencoder.eval()

        # reconstruct test dataset 1
        with pt.no_grad():
            reconstructed_timestep1 = autoencoder(X_test_1[TIMESTEP].unsqueeze(0)).detach().squeeze()

        # reconstruct test dataset 2
        with pt.no_grad():
            reconstructed_timestep2 = autoencoder(X_test_2[TIMESTEP].unsqueeze(0)).detach().squeeze()

        # compute the mean MSE and the corresponding standard deviation for the lowest rank to compare to higher ranks
        if i == 0:
            mean, std = reconstructed_timestep1.mean(), reconstructed_timestep1.std()
            vmin, vmax = mean - 2*std, mean + 2*std
            levels = pt.linspace(vmin, vmax, 120)

            axes[0][i].set_ylabel("Test Dataset 1")
            axes[1][i].set_ylabel("Test Dataset 2")

        cont = axes[0][i].contourf(xx, yy, reconstructed_timestep1, vmin=vmin, vmax=vmax, levels=levels, extend="both")
        cont = axes[1][i].contourf(xx, yy, reconstructed_timestep2, vmin=vmin, vmax=vmax, levels=levels, extend="both")
        axes[0][i].set_title("latent size {}".format(latent_size))
    
    for row in range(2):
        axes[row][i].set_aspect("equal")
        axes[row][i].set_xticklabels([])
        axes[row][i].set_yticklabels([])

    # add seperate subplot for color axis
    fig.subplots_adjust(right=0.9)
    cax = fig.add_axes([0.99, 0.042, 0.03, 0.885])
    cbar = fig.colorbar(cont, cax=cax,label = r"$c_p$")

    fig.tight_layout()
    fig.savefig(join(OUTPUT_PATH, "timestep_reconstruction_VAE.png"), bbox_inches = "tight")

        